<h1 style="text-align: center;">Basics of Mesh Generation</h1>

This Notebook is devoted to the generation of mesh in *femtoscope*. It mainly relies on two Python packages:
- gmsh (4.11.1)
- meshio (4.4.6)

We will see how to generate meshes from scratch using the Gmsh Python API as well as the generation of meshes from pre-written .geo files (Gmsh's built-in scripting language).

**Warning**: be sure to install the above-mentioned version of `meshio` as newer versions might cause errors.

**If you have questions/comments or want to report a bug, please send me an email at <a href="mailto:hugo.levy@onera.fr">hugo.levy@onera.fr</a>**

In [1]:
# Add femtoscope to the path
%reset
import sys
sys.path.append("../") # go to parent dir

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## Physical groups handling in *femtoscope*

*femtoscope* builds on top on the FEM package Sfepy, which is compatible with VTK *(Visualization Toolkit legacy file format)* mesh files. Gmsh allows the definition of *physical groups*, which is a convenient way of grouping model entities. Meshes in *femtoscope* must comply with the following numbering rules:
- vertex entities must be numbered between 0 and 99
- edge entities must be numbered between 100 and 199
- facet entities must be numbered between 200 and 299
- cell entities must be numbered from 300

Please refer to https://sfepy.org/doc-devel/users_guide.html#regions for the definition of *vertex*, *edge*, *facet* and *cell*.

## Mesh generation from scratch

### 1st example: meshing a rectangle

The default numbering of entities in this case will be as follows:

<img src="../images/rectangle_numbering.png" alt="isolated" width="400"/>

Suppose we will not make use of edges 200, 202 and 203. We can ignore these edges (or any other physical entity) with the argument `ignored_tags` passed to the `generate_mesh` method.


In [2]:
from femtoscope.inout.meshfactory import MeshingTools

meshfile = 'rectangle.vtk'
mt = MeshingTools(2)
mt.create_rectangle(xll=0, yll=-1, dx=5, dy=2, centered=False)
mt.create_subdomain(cell_size_min=0.5, cell_size_max=0.5, dist_max=0.0)
mt.generate_mesh(meshfile, show_mesh=True, unique_boundary_tags=False, ignored_tags=[200, 202, 203])

'D:\\hlevy\\Documents\\microscope2\\femtoscope\\tutorials\\..\\data\\mesh\\rectangle.vtk'

The mesh is saved at the path printed just above.
Rectangular meshes are particularly useful when working in polar coordinates $(r, \theta)$ instead of the traditional Cartesian coordinates $(x, y)$. Indeed, a disk in Cartesian coordinates is mapped to a rectangle in polar coordinates. While a circle cannot be perfectly approximated by a finite set of straight line, a rectangle can!

### 2nd example: Multiple cell groups

In [3]:
from femtoscope.inout.meshfactory import MeshingTools

symmetry = False
Ngamma = 200
Rcut = 5.0
sa = 1.0
sc = 0.5

meshfile = 'disk.vtk'
mt = MeshingTools(2)
s1 = mt.create_ellipse(rx=sa, ry=sc)
mt.create_subdomain(cell_size_min=0.03, cell_size_max=0.2, dist_min=0.0, dist_max=4.0)
s2 = mt.create_ellipse(rx=Rcut, ry=Rcut)
mt.subtract_shapes(s2, s1, removeObject=True, removeTool=False)
mt.create_subdomain(cell_size_min=0.2, cell_size_max=0.2)
mt.generate_mesh(meshfile, show_mesh=True, cylindrical_symmetry=True)

'D:\\hlevy\\Documents\\microscope2\\femtoscope\\tutorials\\..\\data\\mesh\\disk.vtk'

Here, we made use of geometry/topology functions such as `create_ellipse` to create the central ellipse, or `subsract_shapes` to substract two topological entities.

## Mesh generation from .geo files

Another way to generate a mesh suitable for *femtoscope* FEM computations is to start from an existing Gmsh geometry file. Such files have the .geo extension and some examples can be found in the directory `data\mesh\geo`. This feature is almost indispensable when dealing with complex geometries.

In the example down below, we pass the argument `param_dic` to the function `mesh_from_geo` in order to directly modify some numerical values of the corresponding .geo file.

In [4]:
from femtoscope.inout.meshfactory import generate_mesh_from_geo

Rcut = 5.0
sa = 1.0
sc = 0.5
generate_mesh_from_geo('mesh_theta.geo', show_mesh=True, param_dict={'size' : 0.1, 'Rc' : Rcut, 'sa' : sa})

'D:\\hlevy\\Documents\\microscope2\\femtoscope\\tutorials\\..\\data\\mesh\\mesh_theta.vtk'

## The specific case of 1D meshes

Contrary to 2D and 3D meshes, 1D meshes are not generated using Gmsh. Instead, we rely on some Sfepy built-in functions to achieve this.

In [5]:
from femtoscope.inout.meshfactory import generate_uniform_1d_mesh

mesh1D = generate_uniform_1d_mesh(0, Rcut, 500 + 1, 'mesh_1d')
print(mesh1D)

Mesh:mesh_1d
  _coors:
    (501, 1) array of float64
  cmesh:
    CMesh: n_coor: 501, dim 1, tdim: 1, n_el 500
  cmesh_tdim:
    list: [None, <sfepy.discrete.common.extmods.cmesh.CMesh object at 0x0000029E7F2EBB00>, None, None]
  descs:
    list: ['1_2']
  dim:
    1
  dims:
    list: [1]
  io:
    None
  n_el:
    500
  n_nod:
    501
  name:
    mesh_1d
  nodal_bcs:
    dict with keys: []


Note that the mesh is not saved in `data\mesh\` but is nonetheless accessible through the variable `mesh1D`.